In [1]:
!pip3 install imutils


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [1]:
import joblib

# Save the model
def save_model(model, path):
    #joblib.dump(svm_clf_dct, 'svm_dct_model.pkl')
    joblib.dump(model, path)
    
def load_model(path):
    # Load the model
    #svm_clf_dct_local = joblib.load('/kaggle/input/svm-dct-model/svm_dct_model.pkl')
    return(joblib.load(path))


# **CLASSICAL METHODS PART**

In [2]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from tqdm import tqdm  # for progress bar
import random 

import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from imutils import paths
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score 
from skimage.filters import gabor
from scipy.spatial.qhull import QhullError
from scipy import spatial
spatial.QhullError = QhullError
from skimage.feature import local_binary_pattern
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


/var/folders/4s/9lp6822x67n5q20pk553klcw0000gn/T/ipykernel_95644/18250641.py:26: DeprecationWarning: Please use `QhullError` from the `scipy.spatial` namespace, the `scipy.spatial.qhull` namespace is deprecated.
  from scipy.spatial.qhull import QhullError


In [3]:
def colortogray(im):
    image = cv2.imread(im)
    imgray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return imgray

def resizeImage(image, size):
    return cv2.resize(image, (size,size))

In [4]:
def extract_features_labels(directory):
    features = []
    labels = []
    for emotion_folder in os.listdir(directory):
        emotion_folder_path = os.path.join(directory, emotion_folder)
        
        # Check if it's a directory
        if os.path.isdir(emotion_folder_path):
            print(f"Processing {emotion_folder} images...")
        for image_file in tqdm(os.listdir(emotion_folder_path)):
                image_path = os.path.join(emotion_folder_path, image_file)
                image = colortogray(image_path)
                image = resizeImage(image, 64)

                fd1 = fd1 =  hog(image, orientations=7, pixels_per_cell=(8, 8),cells_per_block=(4, 4),block_norm= 'L2-Hys' ,transform_sqrt = False)
                label = emotion_folder
                labels.append(label)
                features.append(fd1)
    features = np.array(features)
    labels = np.array(labels)
    return features,labels

In [13]:
from scipy.fftpack import dct, idct

# implement 2D DCT
def dct2_method(a):
    return dct(dct(a.T, norm='ortho').T, norm='ortho')

# implement 2D IDCT
def idct2_method(a):
    return idct(idct(a.T, norm='ortho').T, norm='ortho')

def lbp_method(image, P, R):
    lbp = local_binary_pattern(image, P, R, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, P + 3), range=(0, P + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-7)
    return hist

def gabor_filters_method(image, frequencies):
    responses = []
    for frequency in frequencies:
        real, imag = gabor(image, frequency=frequency)
        magnitude = np.sqrt(real**2 + imag**2)
        responses.append(magnitude.flatten())  # Flatten each response
    combined_response = np.concatenate(responses, axis=0)  # Concatenate flattened responses
    return combined_response

def merge_features(*feature_arrays):
    for i in range(1, len(feature_arrays)):
        if feature_arrays[i].shape[0] != feature_arrays[0].shape[0]:
            raise ValueError("Tüm özellik dizileri aynı sayıda örneğe sahip olmalıdır.")

    # Numpy'ın hstack fonksiyonu ile özellikler yatay olarak birleştirilir
    combined_features = np.hstack(feature_arrays)
    return combined_features

In [10]:
# DISPLAY METHODS

import math
emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

def view_random_images(directory, target_class, num_of_samples=8):
  target_folder = os.path.join(directory, target_class)

  random_images = random.sample(os.listdir(target_folder), num_of_samples)
  print(random_images)
  # Create a subplot to display multiple images
  plt.figure(figsize=(12, 6))
  for i, image in enumerate(random_images):
    plt.subplot(math.ceil(num_of_samples / 4), 4, i + 1)  # 2 rows, 4 columns
    img = cv2.imread(os.path.join(target_folder, image))
    
    plt.imshow(img)
    plt.title(f"{target_class} - {i + 1}")
    plt.axis("off")

  plt.show()

def get_sample_of_images(directory, num_of_samples = 8):    
    for emotion in emotions:
        view_random_images(directory, emotion, num_of_samples)

def get_sample_of_feature_extractions(directory, num_of_samples = 8, method_name = 'dct'):
    for emotion_target in emotions:
        target_folder = os.path.join(directory, emotion_target)
        random_images = random.sample(os.listdir(target_folder), num_of_samples)
        print(emotion_target)
        for i, image in enumerate(random_images):
            img = cv2.imread(os.path.join(target_folder, image))
            print(image)
            print(use_method_on_image(img, method_name))
    
def use_method_on_image(image, method_name = 'dct'):
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img_gray = resizeImage(img_gray, 64)
    
    if method_name == 'dct':
        return dct2_method(img_gray)
    elif method_name == 'idct':
        return idct2_method(img_gray)
    elif method_name == 'lbp':
        return lbp_method(img_gray, P=8, R=1)
    elif method_name == 'gabor':
        return gabor_filters_method(img_gray, frequencies=[0.6, 1.0])
    

def get_random_images_by_emotion(directory, target_class, num_of_samples=8):
    target_folder = os.path.join(directory, target_class)
    random_images = random.sample(os.listdir(target_folder), num_of_samples)
    return random_images
    #images_directory = '/kaggle/input/face-expression-recognition-dataset/images/train'
    #get_random_images_by_emotion(images_directory, 'angry', 8)

In [8]:
images_directory = '/kaggle/input/face-expression-recognition-dataset/images/train'
images_directory = '../Data/images/train'

#get_sample_of_feature_extractions(images_directory, 4, 'gabor')


In [6]:
def extract_features_labels_by_method(directory, method, total_input_count = -1):
    features = []
    labels = []
    for emotion_folder in os.listdir(directory):
        emotion_folder_path = os.path.join(directory, emotion_folder)
        
        if os.path.isdir(emotion_folder_path):
            print(f"Processing {emotion_folder} images...")
            count = 0
            for image_file in tqdm(os.listdir(emotion_folder_path)):
                if total_input_count > 0 and count >= total_input_count:
                    continue
                count += 1
                image_path = os.path.join(emotion_folder_path, image_file)
                img = cv2.imread(image_path)
                features.append(use_method_on_image(img, method))
                label = emotion_folder
                labels.append(label)
    return features, labels

In [13]:
#images_directory = '/kaggle/input/face-expression-recognition-dataset/images/train'
features_dct, labels_dct = extract_features_labels_by_method(images_directory, 'dct')

Processing happy images...


  0%|          | 0/7164 [00:00<?, ?it/s]

100%|██████████| 7164/7164 [00:01<00:00, 5757.31it/s]


Processing sad images...


100%|██████████| 4938/4938 [00:00<00:00, 6004.37it/s]


...

100%|██████████| 4982/4982 [00:00<00:00, 5818.66it/s]


Processing angry images...


100%|██████████| 3993/3993 [00:00<00:00, 5884.25it/s]


Processing disgust images...


100%|██████████| 436/436 [00:00<00:00, 9114.69it/s]


In [9]:
def flatten_images(images):
    flattened_images = []
    for img in images:
        # Flatten the image and append it to the flattened_images list
        flattened_images.append(img.flatten())
    return flattened_images

In [38]:
# DCT MODEL
R =random.randint(1,88)
X_train_dct, X_test_dct, y_train_dct, y_test_dct = train_test_split(features_dct, labels_dct, test_size= 0.3,random_state=R)


X_train_dct_array = np.array(X_train_dct)
nsamples, nx, ny = X_train_dct_array.shape
d2_train_dataset = X_train_dct_array.reshape((nsamples,nx*ny))
print(nsamples, nx, ny)

svm_clf_dct = SVC(kernel='rbf', gamma='scale', C= 10)
svm_clf_dct.fit(d2_train_dataset, y_train_dct)

20174 64 64


SVC(C=10)

In [41]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

def train(model_type, features, labels, test_size=0.2):
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=test_size, random_state=42)
    if model_type == 'svm':
        model = SVC(kernel='rbf', gamma='scale', C= 10)
    elif model_type == 'knn':
        model = KNeighborsClassifier(n_neighbors=5)
    elif model_type == 'rf':
        model = RandomForestClassifier(n_estimators=100, random_state=42)
     
    model.fit(X_train, y_train)
    return model, X_train, X_test, y_train, y_test

def test(model, X_test, y_test):
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)


def predict_and_display(model, image_path):
    # Load and preprocess the image
    img = cv2.imread(image_path)
    img_processed = preprocess_image(img)  # Replace with your actual preprocessing steps

    # Predict the expression
    prediction = model.predict([img_processed])

    # Display the image and prediction
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))  # Convert color from BGR to RGB
    plt.title(f"Predicted Expression: {prediction}")
    plt.axis("off")
    plt.show()

def preprocess_image(image):
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    processed_image = resizeImage(img_gray, 64)
    processed_image = processed_image.flatten()
    return processed_image


def predict_emotion(model, image_path):
    img = cv2.imread(image_path)
    dct_feature = use_method_on_image(img, 'idct')
    # Model expects a 2D array as input; reshape the feature to meet this requirement
    dct_feature_reshaped = dct_feature.reshape(1, -1)
    prediction = model.predict(dct_feature_reshaped)
    return prediction[0]

def output_test_on_image_list(model, images_path):
    for image_path in images_path:
        predict_and_display(model, image_path)



In [21]:
features_idct, labels_idct = extract_features_labels_by_method(images_directory, 'idct', 500)

# Convert features to a suitable shape
features_gabor = np.array([f.flatten() for f in features_idct])
model_gabor_knn, X_train_knn, X_test_knn, y_train_knn, y_test_knn = train('svm', features_gabor, labels_idct)
acc_knn_gabor = test(model_gabor_knn, X_test_knn, y_test_knn)
print(f"KNN Accuracy with Gabor Features: {acc_knn_gabor}")



Processing happy images...


100%|██████████| 7164/7164 [00:00<00:00, 174919.34it/s]


Processing sad images...


100%|██████████| 4938/4938 [00:00<00:00, 156711.58it/s]


Processing fear images...


...

Processing angry images...


100%|██████████| 3993/3993 [00:00<00:00, 123495.60it/s]


Processing disgust images...


100%|██████████| 436/436 [00:00<00:00, 16525.09it/s]


KNN Accuracy with Gabor Features: 0.31976744186046513


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

features_idct, labels_idct = extract_features_labels_by_method(images_directory, 'idct', 200)
features_lbp, labels_idct = extract_features_labels_by_method(images_directory, 'lbp', 200)
features_gabor, labels_idct = extract_features_labels_by_method(images_directory, 'gabor', 200)

features_idct = np.array([f.flatten() for f in features_idct])


features_gabor = np.array(features_gabor)
features_lbp = np.array(features_lbp)

combined_features = merge_features(features_idct, features_lbp, features_gabor)

# Örnek model listesi
models = {
    "KNN": KNeighborsClassifier(),
    "SVM": SVC(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Naive Bayes": GaussianNB(),
}

# Veri setini bölme
X_train, X_test, y_train, y_test = train_test_split(combined_features, labels_idct, test_size=0.2, random_state=32)

# Her modeli eğitin ve test edin
for name, model in models.items():
    model.fit(X_train, y_train)
    predictions_test = model.predict(X_test)
    predictions_train = model.predict(X_train)
    accuracy_test = accuracy_score(y_test, predictions_test)
    accuracy_train = accuracy_score(y_train, predictions_train)
    print(f"{name} Accuracy Train: {accuracy_train}")
    print(f"{name} Accuracy Test: {accuracy_test}")


Processing happy images...


100%|██████████| 7164/7164 [00:00<00:00, 160139.81it/s]


Processing sad images...


100%|██████████| 4938/4938 [00:00<00:00, 136646.26it/s]


Processing fear images...


...

Processing angry images...


100%|██████████| 3993/3993 [00:00<00:00, 36779.21it/s]


Processing disgust images...


100%|██████████| 436/436 [00:00<00:00, 4046.77it/s]


KNN Accuracy Train: 0.43392857142857144
KNN Accuracy Test: 0.18571428571428572
SVM Accuracy Train: 0.35446428571428573
SVM Accuracy Test: 0.2357142857142857
Decision Tree Accuracy Train: 0.9991071428571429
Decision Tree Accuracy Test: 0.18214285714285713
Random Forest Accuracy Train: 0.9991071428571429
Random Forest Accuracy Test: 0.175
Naive Bayes Accuracy Train: 0.45625
Naive Bayes Accuracy Test: 0.26071428571428573


In [31]:
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
import numpy as np
from sklearn.preprocessing import StandardScaler

def feature_selection_and_normalization(features, labels):
    X = np.array(features)
    y = np.array(labels)

    # RFE modeli
    rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=10)
    rfe = rfe.fit(X, y)

    # Seçilen özellikler
    selected_features = rfe.support_

    # Filtrelenmiş özellikler
    X_filtered = X[:, selected_features]
    return X_filtered, y, selected_features
    scaler = StandardScaler()
    X_normalized = scaler.fit_transform(X_filtered)

    return X_normalized, y, selected_features

In [32]:
X_normalized, y, selected_features = feature_selection_and_normalization(combined_features, labels_idct)

KeyboardInterrupt: 

In [ ]:
predict_and_display(model, '../Data/images/train/happy/14.jpg')

# **CNN PART**

In [63]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Veri seti yolu
train_directory = '../Data/images/train'
test_directory = '../Data/images/train'

# Veri artırma ve ön işleme
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Eğitim ve test veri setlerini yükleme
train_generator = train_datagen.flow_from_directory(
    train_directory,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_directory,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical')

# CNN modeli oluşturma
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(7, activation='softmax')  # Sınıf sayısına göre ayarlayın
])

# Modeli derleme
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli eğitme
history = model.fit(train_generator, epochs=10, validation_data=test_generator)




Found 28821 images belonging to 7 classes.
Found 28821 images belonging to 7 classes.
Epoch 1/10
901/901 [==============================] - 24s 27ms/step - loss: 1.6106 - accuracy: 0.3699 - val_loss: 1.4469 - val_accuracy: 0.4417
Epoch 2/10
901/901 [==============================] - 24s 27ms/step - loss: 1.3974 - accuracy: 0.4662 - val_loss: 1.2775 - val_accuracy: 0.5098
Epoch 3/10
901/901 [==============================] - 24s 27ms/step - loss: 1.2851 - accuracy: 0.5106 - val_loss: 1.1930 - val_accuracy: 0.5488
Epoch 4/10
901/901 [==============================] - 23s 26ms/step - loss: 1.1901 - accuracy: 0.5515 - val_loss: 1.0899 - val_accuracy: 0.6011
Epoch 5/10
901/901 [==============================] - 23s 26ms/step - loss: 1.0903 - accuracy: 0.5926 - val_loss: 0.9790 - val_accuracy: 0.6447
Epoch 6/10
901/901 [==============================] - 23s 26ms/step - loss: 0.9929 - accuracy: 0.6297 - val_loss: 0.8727 - val_accuracy: 0.6864
Epoch 7/10
901/901 [==============================

In [64]:
# save the model
save_model(model, 'model.h5')

In [67]:
loaded_model = load_model('model.h5')

print(train_generator.class_indices)

{'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}


In [89]:
emotion_label_list = {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import img_to_array, load_img

def preprocess_image(image_path, target_size=(64, 64)):
    # Görüntüyü yükle ve boyutlandır
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)  # Görüntüyü array'e dönüştür
    img_array = np.expand_dims(img_array, axis=0)  # Modelin beklediği şekle getir
    return img_array / 255.0  # Normalize et

def predict_emotion(model, image_path):
    processed_image = preprocess_image(image_path)
    prediction = model.predict(processed_image)
    return prediction

# Modelinizi yükleyin (eğer henüz yüklenmediyse)
# model = tf.keras.models.load_model('model_yolu')

# # Tahmin yap
# image_path = '../Data/images/train/angry/126.jpg'
# prediction = predict_emotion(model, image_path)

# # Prediction'ı yorumlayın
# # Örneğin, en yüksek skorlu sınıfı bulabilirsiniz:
# predicted_class = np.argmax(prediction, axis=1)
# print(f"Predicted class: {predicted_class}")
# for key, value in emotion_label_list.items():
#     if value == predicted_class:
#         print(key)

# loss, accuracy = model.evaluate(train_generator)
# print(f"Model Accuracy: {accuracy}")

# for random 50 images in ../Data/images/train/angry predict the emotion
predicted_classes = [0,0,0,0,0,0,0]
for image in get_random_images_by_emotion('../Data/images/validation', 'surprise', 50):
    image_path = f'../Data/images/validation/surprise/{image}'
    prediction = predict_emotion(model, image_path)
    predicted_class = np.argmax(prediction, axis=1)
    predicted_classes[int(predicted_class)] += 1

print(predicted_classes)


1/1 [==============================] - 0s 9ms/step
[2, 1, 8, 2, 1, 2, 34]


In [33]:
from tensorflow.keras.layers import Input, concatenate
from tensorflow.keras.models import Model

# Önceki CNN modeliniz
cnn_input = Input(shape=(64, 64, 3))  # CNN için giriş
cnn_model = model(cnn_input)  # CNN modeli

# Geleneksel özellikler için giriş (örneğin, 100 özellik)
traditional_features_input = Input(shape=(100,))

# Özellikleri birleştir
combined_features = concatenate([cnn_model, traditional_features_input])

# Tam bağlantılı katmanlar
x = Dense(64, activation='relu')(combined_features)
output = Dense(7, activation='softmax')(x)  # Sınıf sayısına göre ayarlayın

# Yeni model
new_model = Model(inputs=[cnn_input, traditional_features_input], outputs=output)

# Modeli derle
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


TypeError: 'GaussianNB' object is not callable

In [36]:
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# # Veri seti yolu
# train_directory = '../Data/images/train'
# test_directory = '../Data/images/train'

# # Veri artırma ve ön işleme
# train_datagen = ImageDataGenerator(rescale=1./255)
# test_datagen = ImageDataGenerator(rescale=1./255)

# # Eğitim ve test veri setlerini yükleme
# train_generator = train_datagen.flow_from_directory(
#     train_directory,
#     target_size=(64, 64),
#     batch_size=32,
#     class_mode='categorical')

# test_generator = test_datagen.flow_from_directory(
#     test_directory,
#     target_size=(64, 64),
#     batch_size=32,
#     class_mode='categorical')

# # CNN modeli oluşturma
# model = Sequential([
#     Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
#     MaxPooling2D(2, 2),
#     Conv2D(64, (3, 3), activation='relu'),
#     MaxPooling2D(2, 2),
#     Flatten(),
#     Dense(64, activation='relu'),
#     Dense(7, activation='softmax')  # Sınıf sayısına göre ayarlayın
# ])

# # Modeli derleme
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Modeli eğitme
# history = model.fit(train_generator, epochs=10, validation_data=test_generator)




# Import the necessary libraries
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Load the VGG16 model without the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Veri seti yolu
train_directory = '../Data/images/train'
test_directory = '../Data/images/train'

# Veri artırma ve ön işleme
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Eğitim ve test veri setlerini yükleme
train_generator = train_datagen.flow_from_directory(
    train_directory,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_directory,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical')

# Extract features from the images using the VGG16 model
train_features = base_model.predict(train_generator)
test_features = base_model.predict(test_generator)

# Preprocess the extracted features
train_features = train_features.reshape(train_features.shape[0], -1)
test_features = test_features.reshape(test_features.shape[0], -1)




Found 28821 images belonging to 7 classes.
Found 28821 images belonging to 7 classes.
901/901 [==============================] - 132s 146ms/step


NameError: name 'Sequential' is not defined

In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
# Build the CNN model
model = Sequential([
    Dense(256, activation='relu', input_shape=(train_features.shape[1],)),
    Dropout(0.5),
    Dense(7, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model using the extracted features
history = model.fit(train_features, train_generator.labels, epochs=10, validation_data=(test_features, test_generator.labels))

Epoch 1/10


ValueError: in user code:

    File "/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/compile_utils.py", line 277, in __call__
        loss_value = loss_obj(y_t, y_p, samp